In [1]:
import $ivy.`org.apache.spark::spark-sql:3.2.0`

import $ivy.$                                  

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)

import org.apache.log4j.{Level, Logger}


In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import java.sql.Date
import scala.collection.JavaConverters._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import java.sql.Date

import scala.collection.JavaConverters._

In [4]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("Hello Spark")
                .getOrCreate()

import spark.implicits._
spark.version

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@5b8170e
import spark.implicits._

res3_2: String = "3.2.0"

In [5]:
println(s"spark.version == ${spark.version}")

spark.version == 3.2.0


# RDD

In [6]:
val sc = spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@12b3e70d

In [7]:
val data = Array(1, 2, 3, 4, 5)
val distData = sc.parallelize(data)

data: Array[Int] = Array(1, 2, 3, 4, 5)
distData: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at cmd6.sc:2

In [8]:
val lines = sc.textFile("/opt/spark/README.md").flatMap(_.split(" "))

lines: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[3] at flatMap at cmd7.sc:1

In [9]:
val lineLengths = lines.map(s => s.length)
val totalLength = lineLengths.reduce((a, b) => a + b)

lineLengths: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[4] at map at cmd8.sc:1
totalLength: Int = 3941

In [10]:
val pairs = lines.map(s => (s, 1))
val counts = pairs.reduceByKey((a, b) => a + b)
counts.take(10)

pairs: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[5] at map at cmd9.sc:1
counts: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[6] at reduceByKey at cmd9.sc:2
res9_2: Array[(String, Int)] = Array(
  ("package", 1),
  ("this", 1),
  ("integration", 1),
  ("Python", 2),
  ("cluster.", 1),
  ("its", 1),
  ("[run", 1),
  ("There", 1),
  ("general", 2),
  ("have", 1)
)

# DataFrame

In [11]:
val df = spark.createDataFrame(Array(
    (1, 2.0, "string1", Date.valueOf("2000-01-01")),
    (2, 3.0, "string2", Date.valueOf("2000-02-01")),
    (3, 4.0, "string3", Date.valueOf("2000-03-01"))
))
df

df: DataFrame = [_1: int, _2: double ... 2 more fields]
res10_1: DataFrame = [_1: int, _2: double ... 2 more fields]

In [12]:
df.printSchema

root
 |-- _1: integer (nullable = false)
 |-- _2: double (nullable = false)
 |-- _3: string (nullable = true)
 |-- _4: date (nullable = true)



In [13]:
val schema = StructType(
    StructField("a", IntegerType, nullable = true) ::
    StructField("b", DoubleType, nullable = true) ::
    StructField("c", StringType, nullable = true) ::
    StructField("d", DateType, nullable = true) ::
    Nil)

schema: StructType = StructType(
  StructField("a", IntegerType, true, {}),
  StructField("b", DoubleType, true, {}),
  StructField("c", StringType, true, {}),
  StructField("d", DateType, true, {})
)

In [14]:
val df1 = spark.createDataFrame(sc.parallelize(
    Seq(
        Row(1, 2.0, "string1", Date.valueOf("2000-01-01")),
        Row(2, 3.0, "string2", Date.valueOf("2000-02-01")),
        Row(3, 4.0, "string3", Date.valueOf("2000-03-01"))
    )),
    schema)
df1

df1: DataFrame = [a: int, b: double ... 2 more fields]
res13_1: DataFrame = [a: int, b: double ... 2 more fields]

In [15]:
df1.printSchema

root
 |-- a: integer (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)



In [16]:
df1.dtypes

res15: Array[(String, String)] = Array(
  ("a", "IntegerType"),
  ("b", "DoubleType"),
  ("c", "StringType"),
  ("d", "DateType")
)

In [17]:
df1.show()

+---+---+-------+----------+
|  a|  b|      c|         d|
+---+---+-------+----------+
|  1|2.0|string1|2000-01-01|
|  2|3.0|string2|2000-02-01|
|  3|4.0|string3|2000-03-01|
+---+---+-------+----------+



In [18]:
df1.columns

res17: Array[String] = Array("a", "b", "c", "d")

In [19]:
df1.select("a", "b", "c").describe().show()

+-------+---+---+-------+
|summary|  a|  b|      c|
+-------+---+---+-------+
|  count|  3|  3|      3|
|   mean|2.0|3.0|   null|
| stddev|1.0|1.0|   null|
|    min|  1|2.0|string1|
|    max|  3|4.0|string3|
+-------+---+---+-------+



In [20]:
df1.withColumn("upper_c", upper($"c")).show()

+---+---+-------+----------+-------+
|  a|  b|      c|         d|upper_c|
+---+---+-------+----------+-------+
|  1|2.0|string1|2000-01-01|STRING1|
|  2|3.0|string2|2000-02-01|STRING2|
|  3|4.0|string3|2000-03-01|STRING3|
+---+---+-------+----------+-------+



In [21]:
df1.filter($"a" === 1).show()

+---+---+-------+----------+
|  a|  b|      c|         d|
+---+---+-------+----------+
|  1|2.0|string1|2000-01-01|
+---+---+-------+----------+



In [22]:
val df2 = spark.createDataFrame(List(
    Row("red", "banana", 1, 10),
    Row("blue", "banana", 2, 20),
    Row("red", "carrot", 3, 30),
    Row("blue", "grape", 4, 40),
    Row("red", "carrot", 5, 50),
    Row("black", "carrot", 6, 60),
    Row("red", "banana", 7, 70),
    Row("red", "grape", 8, 80)).asJava,
StructType(Array(
    StructField("color", StringType, nullable = true),
    StructField("fruit", StringType, nullable = true),
    StructField("v1", IntegerType, nullable = true),
    StructField("v2", IntegerType, nullable = true)))
)

df2.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



df2: DataFrame = [color: string, fruit: string ... 2 more fields]

In [23]:
df2.groupBy("color").avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+



In [24]:
df2.write.mode("overwrite").option("header", "true").csv("foo.csv")
spark.read.option("header", "true").csv("foo.csv").show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  6| 60|
| blue|banana|  2| 20|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|  red|banana|  7| 70|
|  red|banana|  1| 10|
|  red|carrot|  3| 30|
|  red| grape|  8| 80|
+-----+------+---+---+



In [25]:
df.write.mode("overwrite").parquet("bar.parquet")
spark.read.parquet("bar.parquet").show()

+---+---+-------+----------+
| _1| _2|     _3|        _4|
+---+---+-------+----------+
|  1|2.0|string1|2000-01-01|
|  3|4.0|string3|2000-03-01|
|  2|3.0|string2|2000-02-01|
+---+---+-------+----------+



In [26]:
df.write.mode("overwrite").orc("zoo.orc")
spark.read.orc("zoo.orc").show()

+---+---+-------+----------+
| _1| _2|     _3|        _4|
+---+---+-------+----------+
|  2|3.0|string2|2000-02-01|
|  3|4.0|string3|2000-03-01|
|  1|2.0|string1|2000-01-01|
+---+---+-------+----------+



In [27]:
df2.createOrReplaceTempView("tableA")
spark.sql("SELECT count(*) from tableA").show()

+--------+
|count(1)|
+--------+
|       8|
+--------+



In [28]:
val add_one = (a: String) => a + 1
val add_oneUDF = udf(add_one)

df2.select(add_oneUDF($"v1").as("add_one_v1")).show

+----------+
|add_one_v1|
+----------+
|        11|
|        21|
|        31|
|        41|
|        51|
|        61|
|        71|
|        81|
+----------+



add_one: String => String = ammonite.$sess.cmd27$Helper$$Lambda$7045/501621666@3da2a059
add_oneUDF: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd27$Helper$$Lambda$7045/501621666@3da2a059,
  StringType,
  List(
    Some(
      ExpressionEncoder(
        StaticInvoke(
          class org.apache.spark.unsafe.types.UTF8String,
          StringType,
          "fromString",
          List(BoundReference(0, ObjectType(class java.lang.String), true)),
          List(),
          true,
          false
        ),
        Invoke(
          UpCast(
            GetColumnByOrdinal(0, StringType),
            StringType,
            List("- root class: \"java.lang.String\"")
          ),
          "toString",
          ObjectType(class java.lang.String),
          List(),
          List(),
          true,
          false
        ),
        java.lang.String
      )
    )
  ),
  Some(
    ExpressionEncoder(
      StaticInvoke(
        class org.apache.spark.unsafe.typ

In [29]:
spark.udf.register("add_one", add_oneUDF)
spark.sql("SELECT add_one(v1) FROM tableA").show()

+-----------+
|add_one(v1)|
+-----------+
|         11|
|         21|
|         31|
|         41|
|         51|
|         61|
|         71|
|         81|
+-----------+



res28_0: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cmd27$Helper$$Lambda$7045/501621666@3da2a059,
  StringType,
  List(
    Some(
      ExpressionEncoder(
        StaticInvoke(
          class org.apache.spark.unsafe.types.UTF8String,
          StringType,
          "fromString",
          List(BoundReference(0, ObjectType(class java.lang.String), true)),
          List(),
          true,
          false
        ),
        Invoke(
          UpCast(
            GetColumnByOrdinal(0, StringType),
            StringType,
            List("- root class: \"java.lang.String\"")
          ),
          "toString",
          ObjectType(class java.lang.String),
          List(),
          List(),
          true,
          false
        ),
        java.lang.String
      )
    )
  ),
  Some(
    ExpressionEncoder(
      StaticInvoke(
        class org.apache.spark.unsafe.types.UTF8String,
        StringType,
        "fromString",
        List(BoundReference(0, Obj

# DataSet

In [30]:
case class Person(name: String, age: Long)

val personDS = Seq(Person("Andy", 32), Person("Mike", 25), Person("Mary", 18)).toDS()

defined class Person
personDS: Dataset[Person] = [name: string, age: bigint]

In [31]:
personDS.show

+----+---+
|name|age|
+----+---+
|Andy| 32|
|Mike| 25|
|Mary| 18|
+----+---+



In [32]:
personDS.printSchema

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = false)



In [33]:
personDS.filter($"age" > 20).show

+----+---+
|name|age|
+----+---+
|Andy| 32|
|Mike| 25|
+----+---+



In [34]:
personDS.filter(_.age > 20).show

+----+---+
|name|age|
+----+---+
|Andy| 32|
|Mike| 25|
+----+---+



In [35]:
personDS.select(col("name")).show

+----+
|name|
+----+
|Andy|
|Mike|
|Mary|
+----+



In [36]:
personDS.select('name).show

+----+
|name|
+----+
|Andy|
|Mike|
|Mary|
+----+



In [37]:
case class PersonCount(name: String, count: Long)

defined class PersonCount

In [38]:
personDS.groupBy('name).count.as[PersonCount]

res37: Dataset[PersonCount] = [name: string, count: bigint]

In [39]:
spark.stop()